# PyCity Schools Analysis 

In [1]:
import pandas as pd
from pathlib import Path
from functools import reduce

In [2]:
# File to load
path_school_data = Path("Resources/schools_complete.csv")
path_student_data = Path("Resources/students_complete.csv")

# Read data files and store into Pandas DataFrame 
school_data = pd.read_csv(path_school_data)
student_data = pd.read_csv(path_student_data)

#combine the data into a single dataset.
complete_school_data = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
complete_school_data.head()


,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [3]:
school_count = len(complete_school_data["school_name"].unique())
school_count

15

In [4]:
student_count = len(complete_school_data["student_name"])
student_count


39170

In [5]:
total_budget = complete_school_data["budget"].unique().sum()
total_budget

24649428

In [6]:
avg_math_score= complete_school_data["math_score"].mean()
avg_math_score

78.98537145774827

In [7]:
avg_reading_score = complete_school_data["reading_score"].mean()
avg_reading_score

81.87784018381414

In [8]:
passing_math_count = complete_school_data.loc[complete_school_data["math_score"] >= 70].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_count) * 100
passing_math_percentage

74.9808526933878

In [9]:
passing_reading_count = complete_school_data.loc[complete_school_data["reading_score"] >= 70].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(student_count) * 100
passing_reading_percentage

85.80546336482001

In [10]:
passing_math_reading_count = complete_school_data.loc[(complete_school_data["math_score"] >= 70) & 
                                                      (complete_school_data["reading_score"] >= 70)].count()["student_name"]
overall_passing_rate = passing_math_reading_count /  float(student_count) * 100
overall_passing_rate 

65.17232575950983

# District Summary

In [11]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
summary_data = [school_count, student_count, total_budget, avg_math_score, avg_reading_score, 
                passing_math_percentage, passing_reading_percentage, overall_passing_rate]

summary_columns = ["Total Schools", "Total Students", "Total Budget","Average Math Score", 
                   "Avrage Reading Score", "% Passing Math" ,"% Passing Reading", "% Overall Passing"]

district_summary = pd.DataFrame([dict(zip(summary_columns, summary_data))])
# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

# Display the DataFrame
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Avrage Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


In [12]:
school_groupby = complete_school_data.groupby("school_name").agg({
  'type': 'unique', 'student_name': 'count',  'budget': 'unique', 'math_score': 'mean', 'reading_score': 'mean'
})

school_groupby

,type,student_name,budget,math_score,reading_score
school_name,,,,,
Bailey High School,[District],4976,[3124928],77.048432,81.033963
Cabrera High School,[Charter],1858,[1081356],83.061895,83.975780
Figueroa High School,[District],2949,[1884411],76.711767,81.158020
Ford High School,[District],2739,[1763916],77.102592,80.746258
Griffin High School,[Charter],1468,[917500],83.351499,83.816757
Hernandez High School,[District],4635,[3022020],77.289752,80.934412
Holden High School,[Charter],427,[248087],83.803279,83.814988
Huang High School,[District],2917,[1910635],76.629414,81.182722
Johnson High School,[District],4761,[3094650],77.072464,80.966394


In [13]:
school_groupby["type"] = school_groupby["type"].apply(lambda x: ','.join(x))
school_groupby["budget"] = school_groupby["budget"].apply(lambda x:int(x[0]))
school_groupby["Per student budget"] = school_groupby["budget"]/school_groupby["student_name"]




In [14]:
math_pass_df = complete_school_data.loc[(complete_school_data["math_score"] >= 70)]
math_pass_groupby = math_pass_df.groupby("school_name").agg({'math_score': 'count'})
reading_pass_df = complete_school_data.loc[(complete_school_data["reading_score"] >= 70)]
reading_pass_groupby = reading_pass_df.groupby("school_name").agg({'reading_score': 'count'})

school_groupby['% passing math'] = math_pass_groupby['math_score']/school_groupby["student_name"]
school_groupby['% passing reading'] = reading_pass_groupby['reading_score']/school_groupby["student_name"]

In [15]:
overall_pass_df = complete_school_data.loc[(complete_school_data["math_score"] >= 70) & (complete_school_data["reading_score"] >= 70)]
overall_pass_groupby = overall_pass_df.groupby('school_name').agg({'reading_score': 'count'})
school_groupby['% Overall Passing'] = overall_pass_groupby['reading_score']/school_groupby["student_name"]

In [16]:
school_spending_df = school_groupby.copy()
school_size_df = school_groupby.copy()
school_type_df = school_groupby.copy()

In [17]:
school_groupby['math_score'] = (school_groupby['math_score']).map("{:.2f}".format)
school_groupby['reading_score'] = (school_groupby['reading_score']).map("{:.2f}".format)
school_groupby['% passing math'] = (school_groupby['% passing math']*100).map("{:.1f}%".format)
school_groupby['% passing reading'] = (school_groupby['% passing reading']*100).map("{:.1f}%".format)
school_groupby['% Overall Passing'] = (school_groupby["% Overall Passing"]*100).map("{:.1f}%".format)

In [18]:
school_groupby.rename(columns={'type' : 'School type','student_name' : 'Total students', "budget" : "Total school budget",
                               'math_score' : 'Average math score', "reading_score" : "Average reading score"
                               }, inplace=True)

In [19]:
per_school_summary = school_groupby
per_school_summary

,School type,Total students,Total school budget,Average math score,Average reading score,Per student budget,% passing math,% passing reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,District,4976,3124928,77.05,81.03,628.0,66.7%,81.9%,54.6%
Cabrera High School,Charter,1858,1081356,83.06,83.98,582.0,94.1%,97.0%,91.3%
Figueroa High School,District,2949,1884411,76.71,81.16,639.0,66.0%,80.7%,53.2%
Ford High School,District,2739,1763916,77.10,80.75,644.0,68.3%,79.3%,54.3%
Griffin High School,Charter,1468,917500,83.35,83.82,625.0,93.4%,97.1%,90.6%
Hernandez High School,District,4635,3022020,77.29,80.93,652.0,66.8%,80.9%,53.5%
Holden High School,Charter,427,248087,83.80,83.81,581.0,92.5%,96.3%,89.2%
Huang High School,District,2917,1910635,76.63,81.18,655.0,65.7%,81.3%,53.5%
Johnson High School,District,4761,3094650,77.07,80.97,650.0,66.1%,81.2%,53.5%


In [20]:
school_groupby = school_groupby.sort_values(by= '% Overall Passing')
bottom_schools = school_groupby.head()
bottom_schools

,School type,Total students,Total school budget,Average math score,Average reading score,Per student budget,% passing math,% passing reading,% Overall Passing
school_name,,,,,,,,,
Rodriguez High School,District,3999,2547363,76.84,80.74,637.0,66.4%,80.2%,53.0%
Figueroa High School,District,2949,1884411,76.71,81.16,639.0,66.0%,80.7%,53.2%
Hernandez High School,District,4635,3022020,77.29,80.93,652.0,66.8%,80.9%,53.5%
Huang High School,District,2917,1910635,76.63,81.18,655.0,65.7%,81.3%,53.5%
Johnson High School,District,4761,3094650,77.07,80.97,650.0,66.1%,81.2%,53.5%


In [21]:
avg_math_score = complete_school_data.groupby(['school_name', 'grade'])['math_score'].mean().reset_index()
pivot1 = avg_math_score.pivot_table(index='school_name', columns='grade', values='math_score')
pivot1

grade,10th,11th,12th,9th
school_name,,,,
Bailey High School,76.996772,77.515588,76.492218,77.083676
Cabrera High School,83.154506,82.765560,83.277487,83.094697
Figueroa High School,76.539974,76.884344,77.151369,76.403037
Ford High School,77.672316,76.918058,76.179963,77.361345
Griffin High School,84.229064,83.842105,83.356164,82.044010
Hernandez High School,77.337408,77.136029,77.186567,77.438495
Holden High School,83.429825,85.000000,82.855422,83.787402
Huang High School,75.908735,76.446602,77.225641,77.027251
Johnson High School,76.691117,77.491653,76.863248,77.187857


In [22]:
avg_reading_score = complete_school_data.groupby(['school_name', 'grade'])['reading_score'].mean().reset_index()
pivot2 = avg_reading_score.pivot_table(index='school_name', columns='grade', values='reading_score')
pivot2

grade,10th,11th,12th,9th
school_name,,,,
Bailey High School,80.907183,80.945643,80.912451,81.303155
Cabrera High School,84.253219,83.788382,84.287958,83.676136
Figueroa High School,81.408912,80.640339,81.384863,81.198598
Ford High School,81.262712,80.403642,80.662338,80.632653
Griffin High School,83.706897,84.288089,84.013699,83.369193
Hernandez High School,80.660147,81.396140,80.857143,80.866860
Holden High School,83.324561,83.815534,84.698795,83.677165
Huang High School,81.512386,81.417476,80.305983,81.290284
Johnson High School,80.773431,80.616027,81.227564,81.260714


In [23]:
spending_bins = [0, 585, 630, 645, 680]
labels = ["$000-$585", "$585-630", "$630-645", "$645-680"]
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(school_spending_df['Per student budget'], bins=spending_bins, labels=labels, include_lowest=True)

In [24]:
school_spending_df.rename(columns={'math_score' : 'Average math score', "reading_score" : "Average reading score"}, inplace=True)


In [25]:
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average math score"].mean().reset_index()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average reading score"].mean().reset_index()
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% passing math"].mean().reset_index()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% passing reading"].mean().reset_index()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Overall Passing"].mean().reset_index()

In [26]:
spending_data =[spending_reading_scores, spending_passing_math,
               spending_passing_reading, overall_passing_spending]

spending_summary= spending_math_scores

for frame in spending_data:
    spending_summary = spending_summary.merge(frame, how='right')

In [27]:
spending_summary

,Spending Ranges (Per Student),Average math score,Average reading score,% passing math,% passing reading,% Overall Passing
0,$000-$585,83.455399,83.933814,0.934601,0.966109,0.903695
1,$585-630,81.899826,83.155286,0.871335,0.927182,0.814186
2,$630-645,78.518855,81.624473,0.734842,0.843918,0.628577
3,$645-680,76.997210,81.027843,0.661648,0.811340,0.535269


In [28]:
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
school_size_df["school size"] = pd.cut(school_size_df['student_name'], bins=size_bins, labels=labels, include_lowest=True)
school_size_df.rename(columns={'math_score' : 'Average math score', "reading_score" : "Average reading score"}, inplace=True)


In [29]:
size_math_scores = school_size_df.groupby(["school size"])["Average math score"].mean().reset_index()
size_reading_scores = school_size_df.groupby(["school size"])["Average reading score"].mean().reset_index()
size_passing_math = school_size_df.groupby(["school size"])["% passing math"].mean().reset_index()
size_passing_reading = school_size_df.groupby(["school size"])["% passing reading"].mean().reset_index()
size_overall_passing = school_size_df.groupby(["school size"])["% Overall Passing"].mean().reset_index()

In [30]:
size_data =[size_reading_scores, size_passing_math,
               size_passing_reading, size_overall_passing]

size_summary= size_math_scores

for frame in size_data:
    size_summary = size_summary.merge(frame, how='right')

size_summary

,school size,Average math score,Average reading score,% passing math,% passing reading,% Overall Passing
0,Small (<1000),83.821598,83.929843,0.935502,0.960994,0.898839
1,Medium (1000-2000),83.374684,83.864438,0.935997,0.967907,0.906215
2,Large (2000-5000),77.746417,81.344493,0.699634,0.827666,0.582860


In [31]:
school_type_df.rename(columns={'type' : 'school type', 'math_score' : 'Average math score', "reading_score" : "Average reading score"}, inplace=True)

In [32]:
type_math_scores = school_type_df.groupby(["school type"])["Average math score"].mean().reset_index()
type_reading_scores = school_type_df.groupby(["school type"])["Average reading score"].mean().reset_index()
type_passing_math = school_type_df.groupby(["school type"])["% passing math"].mean().reset_index()
type_passing_reading = school_type_df.groupby(["school type"])["% passing reading"].mean().reset_index()
type_overall_passing = school_type_df.groupby(["school type"])["% Overall Passing"].mean().reset_index()

In [33]:
type_data =[type_reading_scores, type_passing_math,
               type_passing_reading, type_overall_passing]

type_summary= type_math_scores

for frame in type_data:
    type_summary = type_summary.merge(frame, how='right')

type_summary

,school type,Average math score,Average reading score,% passing math,% passing reading,% Overall Passing
0,Charter,83.473852,83.896421,0.936208,0.965865,0.904322
1,District,76.956733,80.966636,0.665485,0.807991,0.536722
